<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Pandas per la gestione dei dati
import pandas as pd

from collections import Counter

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# Importiamo il dataset ottenuto tramite le operazioni di scraping
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/charting-m-points-2010s.csv")

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329842,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,119,0,1,3,5.0,AD-40,19,True,1,4d,4b3n#,NaN,1
329843,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,120,0,1,4,5.0,0-0,20,True,2,5d,6f1w#,NaN,2
329844,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,121,0,1,4,5.0,15-0,20,True,2,5s38b2f1f1f2b2b2f1f3b3b+1n@,NaN,NaN,2
329845,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,122,0,1,4,5.0,30-0,20,True,2,4f18f1f3b3s3b3s3f+1l1o3*,NaN,NaN,2


Analisi delle tipologie degli esiti finali dei punti, analisi dei casi rari

In [26]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 110288
'@': 103774
'*': 102804
'd': 4794
'n': 4777
'w': 1376
'S': 1003
'R': 429
'x': 264
'C': 135
';': 92
'!': 22
' ': 16
'g': 14
'Q': 12
'1': 10
'3': 9
'e': 6
'P': 5
'9': 4
'2': 2
'c': 2
'7': 2
'p': 1
'^': 1
'+': 1
'.': 1
'-': 1
'8': 1
'b': 1


Correzione dei casi rari in un formato più gestibile

In [27]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):

    ultimo_carattere = row['1st'][-1]
    simboli_finali[ultimo_carattere] += 1




    # Check aces
    numeri = [c for c in row['1st'] if c.isdigit()]
    if len(numeri) == 1:
      #Inserire tutti i casi di aces
      row['1st'][-1]

    else
      #Inserire tutti i casi di non aces
      row['1st'][-1]



  else:

    # Check aces
    numeri = [c for c in row['2nd'] if c.isdigit()]
    if len(numeri) == 1:
      #Inserire tutti i casi di aces
      row['2nd'][-1]

    else
      #Inserire tutti i casi di non aces
      row['2nd'][-1]





for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

Output streaming troncato alle ultime 5000 righe.
6s#
4*
4r#
6*
4*
6*
6s#
4s#
6*
6b#
6*
4s#
4*
6*
6*
4*
6#
6*
6#
4*
6*
6+*
0q#
4+*
6#
4*
4+*
4#
4*
6#
6#
6*
4*
6*
6*
6*
6#
6#
4*
4*
6#
4*
4#
4+*
4*
4*
4#
6*
6*
4*
6*
4*
4*
6*
6*
6*
6#
6*
6*
4*
4*
6*
4*
6*
4*
6r#
6s#
6*
cc6f#
6*
4s#
6s#
4*
6*
c4b#
4b#
6s#
5b#
4f#
4*
4*
4*
6*
4*
6*
6*
6*
4b#
5f#
4*
6*
4f#
6*
6*
6*
6*
6*
6*
6*
6*
4b#
6*
4*
6*
4*
4*
6*
6*
6*
6*
6*
6*
4b#
6*
4*
4*
6*
6*
4*
6*
6*
4*
4b#
6*
6*
6rn#
6*
6s#
6*
6*
6*
4s#
6*
6*
5bn#
4*
5+b#
4fn#
6*
6*
4*
4*
6fn#
4*
6s#
4*
c6*
4*
4*
6*
4*
4*
6*
6*
6*
6*
4*
4*
4*
6f#
4*
4b#
6f#
6#
6s#
4f#
6*
4*
6*
6f#
6f#
4#
6*
6*
6*
6*
4*
6s#
4s#
4*
4b#
4r#
6b#
6*
5f#
6#
6f#
6*
4*
4b#
4b#
6*
6*
4*
6*
4*
6*
6*
6*
6f#
5b#
6*
4*
4*
4*
6*
6*
6*
4*
6*
4*
6#
6*
6*
4*
4*
c4#
6*
6*
6#
6#
c4#
4#
6*
4#
4#
6*
4*
6#
6*
6*
4r#
6*
4f#
4*
4f#
4*
4*
4*
6*
4*
4*
4*
4s#
4*
4*
6*
6*
6*
6*
6*
6*
6*
6*
4*
6*
4*
4*
4*
6*
6*
4f#
6s#
4*
4s#
6b#
6s#
6*
6b#
4f#
4*
4*
4*
6*
4*
6*
6*
4*
4*
4*
6*
c4*
6*
4*
4*
6*
6*
6*
6*
4s#
6*


Creazione nuovo dataset
E decidere come dwefinire le sequenze di punteggi
direi di fare due sequenze, una semplice e una complessa

In [11]:
new_data = []
current_id = ""
current_set = ""
current_game = ""


# Contatore dei simboli
simboli_finali = Counter()


for index, row in dataset.iterrows():
  if row['match_id'] != current_id:
    current_id = row['match_id']

    new_row = {
        'match_id': row['match_id'],
        'PtsProgression':[],
        'GmProgression':[],
        'Set1': row['Set1'],
        'Set2': row['Set2'],
        'Gm1': row['Gm1'],
        'Gm2': row['Gm2'],
        'GmWinner' : 0,
        'SetWinner' : 0
      }

  else:

    if row['Set1']+row['Set2'] != current_set:
      current_set = row['Set1'] + row['Set2']
      new_row = {
        'match_id': row['match_id'],
        'PtsProgression':[],
        'GmProgression':[],
        'Set1': row['Set1'],
        'Set2': row['Set2'],
        'Gm1': row['Gm1'],
        'Gm2': row['Gm2'],
        'GmWinner' : 0,
        'SetWinner' : 0
      }

    else:

      if row['Gm1']+row['Gm2'] != current_game:
        current_game = row['Gm1'] + row['Gm2']

        new_row = {
        'match_id': row['match_id'],
        'PtsProgression':[],
        'GmProgression':[],
        'Set1': row['Set1'],
        'Set2': row['Set2'],
        'Gm1': row['Gm1'],
        'Gm2': row['Gm2'],
        'GmWinner' : 0,
        'SetWinner' : 0
        }

        if(pd.isna(row['2nd'])):
          ultimo_carattere = row['1st'][-1]
          simboli_finali[ultimo_carattere] += 1
        else:
          ultimo_carattere = row['2nd'][-1]
          simboli_finali[ultimo_carattere] += 1





      new_data.append(new_row)
for simbolo, conteggio in simboli_finali.items():
    print(f"'{simbolo}': {conteggio}")
new_dataset = pd.DataFrame(new_data)


'C': 20
'@': 16850
'*': 16133
'#': 17853
'n': 795
'd': 810
'w': 200
';': 9
'S': 269
'R': 84
'Q': 4
'P': 1
'3': 1
'p': 1
'x': 28
'c': 1
' ': 4
'g': 1
'!': 5
'9': 1
'^': 1
'1': 1
'e': 1
